In [1]:
from CCDPApy.helper import input_path
from CCDPApy import FedBatchParameters, FedBatchCellCulture, FedBatchCellLine, FedBatchExpriment
from CCDPApy import cell_culture_pipeline, CellCulturePipeline

import pandas as pd
pd.set_option('display.max_columns', 200)
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interactive, widgets
import seaborn as sns

# Plotly

## code

In [2]:
param_1 = FedBatchParameters(cell_line_name='CL1',
                             use_concentration_after_feed=False,
                             use_feed_concentration=True,
                             regression_method=['polynomial', 'rolling_window_polynomial'],
                             rolling_polynomial_degree=3,
                             rolling_polynomial_window=6)

file = 'fed_batch_data.xlsx'
fed_batch = cell_culture_pipeline(cell_culture_type='fed-batch', file=file)

fed_batch.perform_data_process([param_1])

## plot

In [3]:
fed_batch.interactive_plot(port=8888)

Dash is running on http://127.0.0.1:8888/



# Ipywidjets

## code

In [4]:
culture_handler = FedBatchCellCulture()
culture_handler.load_data(file=input_path('fed_batch_data.xlsx'))
culture_data = culture_handler.get_pre_process_data()

cl_name, cl_id = 'CL1', '11'
use_feed_conc, use_conc_after_feed = True, False

cell_line_handler = FedBatchCellLine(cell_line_name=cl_name, data=culture_data)
cell_line_data = cell_line_handler.get_pre_process_data()

exp_handler = FedBatchExpriment(cell_line_name=cl_name, cell_line_id=cl_id, data=cell_line_data,
                                use_feed_conc=use_feed_conc, use_conc_after_feed=use_conc_after_feed)

exp_handler.in_process()
exp_handler.polynomial()
exp_handler.rolling_window_polynomial(degree=3, window=6)

In [5]:
# Get data
df1 = exp_handler.conc
df2 = exp_handler.cumulative_conc
df3 = exp_handler.sp_rate

# Options
spc_list = list(exp_handler.get_species().keys())
spc_list.remove('cell')
spc_list.remove('product')
spc_list = [(s.upper(), s.capitalize()) for s in spc_list]
date_list = ['Run Time (hr)', 'Run Time (day)']
profile_list = [
    ('Concentration', 1), 
    ('Cumulative Production/Consumption', 2), 
    ('SP. Rate', 3), 
    ('All Profiles', 4)]
method_list = df3['method'].unique()

df = {1: df1, 2: df2, 3: df3}
titles = {1: 'Concentration Profile', 2: 'Cumulative Profile', 3: 'SP. Rate Profile'}

sns.set_style("whitegrid", {'grid.linestyle': ':'})

spc_btn = widgets.Dropdown(
    options=spc_list, 
    description="Species")

spc_btn = widgets.SelectMultiple(
    options=spc_list,
    # value=[spc_list[0][1]],
    rows=3,
    description='Species',
    disabled=False
)

profile_btn = widgets.ToggleButtons(
    options=profile_list,
    description='Profiles:',
    disabled=False,
    button_style=''
)

date_btn = widgets.ToggleButtons(
    options=date_list, 
    description="Run Time:")

height_bar = widgets.IntSlider(
    value=3,
    min=1,
    max=20,
    step=1,
    description='Height:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

width_bar = widgets.IntSlider(
    value=5,
    min=1,
    max=20,
    step=1,
    description='Width:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

def plot_profile(spc, profiles, date, height, width):
    n_spc = len(spc)
    fig_size = (width*3, height*n_spc)

    if n_spc!=0:
        if profiles==4:
            # Create multiple subplots
            fig, axes = plt.subplots(n_spc, 3, figsize=fig_size, tight_layout=True, squeeze=False)
            for i, s in enumerate(spc):
                # Concentration
                ax = axes[i, 0]
                mask = df[1]['species']==s
                sns.lineplot(ax=ax, data=df1[mask], x=date, y='value', estimator=None, marker='o')
                ax.set_title(f"{s} {titles[1]}")
                ax.set_ylabel(f"{s} {df1[mask]['unit'].values[0]}")

                # Cumulative
                ax = axes[i, 1]
                mask = df[2]['species']==s
                data = df2[mask]
                # two-point
                data1 = data[data['method']=='twoPoint']
                sns.scatterplot(ax=ax, data=data1, x=date, y='value', estimator=None)
                # polynomial
                if 'polynomial' in data['method'].unique():
                    data2 = data[data['method']=='polynomial']
                    sns.lineplot(ax=ax, data=data2, x=date, y='value', estimator=None)

                ax.set_title(f"{s} {titles[2]}")
                ax.set_ylabel(f"{s} {df2[mask]['state'].values[0]} {df2[mask]['unit'].values[0]}")
                ax.legend(data['method'].unique())

                # SP. rate
                ax = axes[i, 2]
                mask = df[3]['species']==s
                sns.lineplot(ax=ax, data=df3[mask], x=date, y='value', estimator=None, hue='method', style='method', marker='o')
                ax.set_title(f"{s} {titles[3]}")
                ax.set_ylabel(f"{s} {df3[mask]['unit'].values[0]}")
            
        else:
            # Create plots
            fig, axes = plt.subplots(n_spc, 1, figsize=(width, height*n_spc), tight_layout=True, squeeze=False)
            axes = axes.flatten()
            df_profile = df[profiles]
            for i, s in enumerate(spc):
                ax = axes[i]
                data = df_profile[df_profile['species']==s]
                ylabel = f"{s} {data['unit'].values[0]}"
                if profiles==1: # Concentration
                    sns.lineplot(ax=ax, data=data, x=date, y='value', estimator=None, marker='o')
                elif profiles==2:
                    # two-point
                    data1 = data[data['method']=='twoPoint']
                    sns.scatterplot(ax=ax, data=data1, x=date, y='value', estimator=None)
                    # polynomial
                    if 'polynomial' in data['method'].unique():
                        data2 = data[data['method']=='polynomial']
                        sns.lineplot(ax=ax, data=data2, x=date, y='value', estimator=None)
                    ax.legend(data['method'].unique())
                    ylabel = f"{s} {data['state'].values[0]} {data['unit'].values[0]}"

                elif profiles==3:
                    sns.lineplot(ax=ax, data=data, x=date, y='value', estimator=None, hue='method', style='method', marker='o')
            
                # layout
                ax.set_ylabel(ylabel)
                ax.set_title(f"{s} {titles[profiles]}")
        #plt.savefig('/Users/galileo/Desktop/test.png')
        plt.show()

interactive_plot = interactive(
    plot_profile, 
    spc=spc_btn, 
    profiles=profile_btn,
    date=date_btn, 
    height=height_bar, 
    width=width_bar)

## plot

In [6]:
interactive_plot

interactive(children=(SelectMultiple(description='Species', options=(('ALANINE', 'Alanine'), ('ARGININE', 'Arg…